# Imports

In [ ]:
from pathlib import Path
import pandas as pd, numpy as np
import plotly.express as px
from datetime import datetime

In [ ]:
csvs = list(Path("/Users/alexpayne/lilac-mount-point/asap-datasets/mers_fauxalysis/20230425_docked_for_fauxalysis").glob("run_docking_oe.*-results.csv"))

In [ ]:
len(csvs)

## loadall csvs

In [ ]:
print("start", datetime.now())
df = pd.concat(map(pd.read_csv, csvs), ignore_index= True)
print("end", datetime.now())

In [ ]:
from asapdiscovery.docking.analysis import DockingResults

In [ ]:
dr = DockingResults(df=df, column_names="None")

In [ ]:
dr.df.columns

In [ ]:
dr.df.columns = ["Compound_ID", 
                 "Structure_Source", 
                 "Docked_File", 
                 "Pose_ID", 
                 "RMSD", 
                 "POSIT", 
                 "POSIT_Method", 
                 "Chemgauss4", 
                 "Clash", 
                 "SMILES", 
                 "GAT_Score"]

In [ ]:
dr.df["POSIT_R"] = 1-dr.df["POSIT"]
dr.df["Complex_ID"] = dr.df.Compound_ID.apply(str) + "_" + dr.df.Structure_Source

In [ ]:
structure_fns = list(Path("/Users/alexpayne/lilac-mount-point/asap-datasets/mers_fauxalysis/20230308_prepped_mers_pdbs/").glob("*prepped_receptor_0.pdb"))

In [ ]:
lilac_path = Path("/lila/data/chodera")

In [ ]:
final = lilac_path.joinpath(*structure_fns[0].parts[4:])

In [ ]:
final

In [ ]:
paths = [lilac_path.joinpath(*structure_fn.parts[4:]) for structure_fn in structure_fns]

In [ ]:
structure_paths = []
for structure in dr.df["Structure_Source"].unique():
    for path in paths:
        if structure in str(path):
            structure_paths.append(path)

In [ ]:
structure_source_df = pd.DataFrame({"Structure_Source": dr.df["Structure_Source"].unique(), "Structure_Path": structure_paths})

In [ ]:
dr.df = pd.merge(dr.df, structure_source_df, on="Structure_Source")

In [ ]:
dr.get_best_structure_per_compound(score_order=["POSIT_R", "Chemgauss4", "RMSD"], score_ascending=[True, True, True])

In [ ]:
dr.best_df

In [ ]:
dr.get_compound_df()

In [ ]:
dr.get_structure_df()

# Write analyzed results

In [ ]:
output_dir = Path("/Users/alexpayne/lilac-mount-point/asap-datasets/mers_fauxalysis/20230425_docked_for_fauxalysis")

In [ ]:
dr.write_dfs_to_csv(output_dir=output_dir)